<a href="https://colab.research.google.com/github/lacastanop/se-alesys/blob/main/Dashboard_Parcial_3_2024_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [111]:
#instalación de librerías
!pip install streamlit -q
!pip install pyngrok

In [112]:
#Se puede usar la API youtube-dl para descargar un video de youtube y extraer el audio en formato mp3.
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz -q

#Libreria para manipulación de archivos de audio
!pip install soundfile -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [113]:
!pip install sympy --upgrade

In [115]:
%%writefile 1_📚_Parcial_3.py

import streamlit as st

st.set_page_config(
    page_title="Parcial #3",
    page_icon="👋",
)

st.write("# Bienvenido al Dashboard del Parcial #3! 👋")

st.sidebar.success("Selecciona el tema que deseas explorar.")

st.markdown(
    """
    **👈 Animate y selecciona el tema que sea de tu interes a estudiar**
"""
)

Overwriting 1_📚_Parcial_3.py


In [116]:
!mkdir pages

mkdir: cannot create directory ‘pages’: File exists


In [117]:
%%writefile pages/📈_Pregunta_1.py
import sympy as sym
import matplotlib.pyplot as plt
import sympy.physics.control as control
import streamlit as st

# Inicialización de la impresión de expresiones simbólicas
sym.init_printing()

# Definir las variables simbólicas
s = sym.symbols('s', complex=True)  # Variable de Laplace
t, R, L, C = sym.symbols('t R L C', positive=True)  # Tiempo y parámetros del circuito RLC

# Entrada de interés en Laplace
X = 1/s  # Ajustar según el tipo de entrada (por ejemplo, entrada escalón)

# Definir la función de transferencia para el circuito RLC
H = control.TransferFunction(1, L*C*s**2 + R*C*s + 1, s)  # Función de transferencia del circuito

# Multiplicamos la función de transferencia por la entrada
Y = H.num * X / H.den

# Función de salida en Laplace factorizada (fracciones parciales)
Yp = Y.apart(s)  # Factorización de fracciones parciales
st.write("Función de salida factorizada en fracciones parciales (Laplace):")
st.latex(sym.latex(Yp))

# Modelo en el tiempo en función de los componentes RLC
Ye = Yp
yt = 0  # Iniciar la variable para la solución temporal
for j in range(len(Ye.args)):
    if j == 0:
        yt = sym.inverse_laplace_transform(Ye.args[j], s, t)
    else:
        yt += sym.inverse_laplace_transform(Ye.args[j], s, t)

st.write("\nFunción de salida en el tiempo (solución general):")
st.latex(sym.latex(yt))

# Evaluar la función en valores numéricos para R, L, C
RLC = {R: 1, L: sym.Rational('.5'), C: sym.Rational('.4')}  # Valores utilizados para la simulación
Ye_evaluated = Yp.subs(RLC)

# Calcular la función inversa de Laplace con los valores numéricos
yt_evaluated = 0
for j in range(len(Ye_evaluated.args)):
    if j == 0:
        yt_evaluated = sym.inverse_laplace_transform(Ye_evaluated.args[j], s, t)
    else:
        yt_evaluated += sym.inverse_laplace_transform(Ye_evaluated.args[j], s, t)

st.write("\nFunción de salida en el tiempo con valores evaluados para R, L, C:")
st.latex(sym.latex(yt_evaluated))

# Diagrama de polos y ceros utilizando la API de control de Sympy
HRLC = control.TransferFunction(H.num, H.den.subs(RLC), s)  # Función de transferencia evaluada

# Crear la figura del diagrama de polos y ceros
fig, ax = plt.subplots()
control.pole_zero_plot(HRLC, ax=ax)

# Mostrar la gráfica en Streamlit
st.pyplot(fig)

# Mostrar el diagrama de Bode
fig2 = plt.figure()
control.bode_plot(HRLC)
st.pyplot(fig2)

# Mostrar la respuesta al impulso
fig3 = plt.figure()
control.impulse_response_plot(HRLC)
st.pyplot(fig3)

# Mostrar la respuesta al escalón
fig4 = plt.figure()
control.step_response_plot(HRLC)
st.pyplot(fig4)

Overwriting pages/📈_Pregunta_1.py


In [118]:
!python3 -m pip install --force-reinstall https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz

  Using cached https://github.com/yt-dlp/yt-dlp/archive/master.tar.gz (2.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Brotli-1.1.0-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (5.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached mutagen-1.47.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached pycryptodomex-3.21.0-cp36-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached urllib3-2.2.3-py3-none-any.whl.metadata (6.5 kB)
  Using cached websockets-13.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (33 kB)
  Usi

In [133]:
import os
import yt_dlp as youtube_dl
import subprocess

def download_ytvid_as_mp3(video_url, name):
    # Define el directorio de salida en /content
    output_dir = "/content/"
    filename = f"{output_dir}{name}.mp3"
    options = {
        'format': 'bestaudio/best',
        'keepvideo': False,
        'outtmpl': filename,
    }

    with youtube_dl.YoutubeDL(options) as ydl:
        ydl.download([video_url])

    print(f"Download complete: {filename}")
    #30 canciones, 10 de cada género
#Vallenato
link_= ["https://www.youtube.com/watch?v=itquaX8gH34",
       "https://www.youtube.com/watch?v=EAm1d6I_iDw",
       "https://www.youtube.com/watch?v=SW2aJ7NH3is",
       "https://www.youtube.com/watch?v=SKV6zQyeJeo",
       "https://www.youtube.com/watch?v=tIZCwqx8bUE",
       "https://www.youtube.com/watch?v=30OtPy7PW0g",
       "https://www.youtube.com/watch?v=FU-7mHUTJKk",
       "https://www.youtube.com/watch?v=8GHExewfixM",
       "https://www.youtube.com/watch?v=1rFNoZaxhG4",
       "https://www.youtube.com/watch?v=elCjRzvm5Hs",
#Reggae
        "https://www.youtube.com/watch?v=a5fHoAx12DY",
        "https://www.youtube.com/watch?v=odQRCrCFW04",
        "https://www.youtube.com/watch?v=lWKaMeJRiik",
        "https://www.youtube.com/watch?v=M3geA7B4Y2Q",
        "https://www.youtube.com/watch?v=QuNz2b8RUp4",
        "https://www.youtube.com/watch?v=v788XXX8qdk",
        "https://www.youtube.com/watch?v=TBapnUC865M",
        "https://www.youtube.com/watch?v=oyRFUI8Qg0Q",
        "https://www.youtube.com/watch?v=AQ4TW5Z8eu0",
        "https://www.youtube.com/watch?v=dW2MmuA1nI4",

#Ranchera
        "https://www.youtube.com/watch?v=_p-W0icDaSY",
        "https://www.youtube.com/watch?v=gfm2zSgQ8cQ",
        "https://www.youtube.com/watch?v=xa2GLqL12Zk",
        "https://www.youtube.com/watch?v=YOIavOWa6po",
        "https://www.youtube.com/watch?v=hS5iEP8SUKw",
        "https://www.youtube.com/watch?v=PVb6IHmsQX4",
        "https://www.youtube.com/watch?v=rKvAnlRgBs0",
        "https://www.youtube.com/watch?v=ba3Aq_dBiUc",
        "https://www.youtube.com/watch?v=Xl1Kr9BFuK0",
        "https://www.youtube.com/watch?v=wVXLd8G9-JM"
       ]
name_ = ['Que vuelva',
         'Que No Me Faltes Tú',
         'Busca Un Confidente',
         'Amarte Más No Pude',
         'Tú Eres La Reina',
         'Ese Muchacho',
         'La Suerte Está Echada',
         'Me Vas A Extrañar',
         'Quiero Que Seas MI Estrella',
         'Me Ilusioné',
         'Tú Sin Mí',
         'Nada',
         'Árbol Sin Hojas',
         'Sálvame',
         'Morena',
         'Comando',
         'Good Bye',
         'Agarra Mi Mano',
         "I'm Still In Love With You",
         'Temperature',
         'Un Millón de Primaveras',
         'Por Tu Maldito Amor',
         'Mujeres Divinas',
         'Qué de Raro Tiene',
         'Señor',
         'Maria De Los Guardias',
         'De Carne O Hueso',
         'Ángel O Demonio',
         'Si Supieras',
         'Unas Copas De Mas']

# Descargar todas las canciones
for name, link in zip(name_, link_):
    download_ytvid_as_mp3(link, name)

# Convertir los archivos MP3 a WAV
for name in name_:
    mp3_file = f"/content/{name}.mp3"
    wav_file = f"/content/{name}.wav"

    # Verifica si el archivo MP3 existe antes de convertir
    if os.path.exists(mp3_file):
        subprocess.call(['ffmpeg', '-y', '-i', mp3_file, wav_file])
        print(f"Conversion complete: {wav_file}")
    else:
        print(f"MP3 file not found: {mp3_file}")

[youtube] Extracting URL: https://www.youtube.com/watch?v=itquaX8gH34
[youtube] itquaX8gH34: Downloading webpage
[youtube] itquaX8gH34: Downloading ios player API JSON
[youtube] itquaX8gH34: Downloading web creator player API JSON
[youtube] itquaX8gH34: Downloading m3u8 information
[info] itquaX8gH34: Downloading 1 format(s): 251
[download] Destination: /content/Que vuelva.mp3
[download] 100% of    4.67MiB in 00:00:00 at 14.30MiB/s  
Download complete: /content/Que vuelva.mp3
[youtube] Extracting URL: https://www.youtube.com/watch?v=EAm1d6I_iDw
[youtube] EAm1d6I_iDw: Downloading webpage
[youtube] EAm1d6I_iDw: Downloading ios player API JSON
[youtube] EAm1d6I_iDw: Downloading web creator player API JSON
[youtube] EAm1d6I_iDw: Downloading m3u8 information
[info] EAm1d6I_iDw: Downloading 1 format(s): 251
[download] Destination: /content/Que No Me Faltes Tú.mp3
[download] 100% of    4.56MiB in 00:00:00 at 14.79MiB/s  
Download complete: /content/Que No Me Faltes Tú.mp3
[youtube] Extracting

In [122]:
#Libreria para manipulación de archivos de audio
!pip install soundfile

In [124]:
%%writefile pages/🎶_Pregunta_2.py
import streamlit as st
import soundfile as sf
import numpy as np
import subprocess
import json
import matplotlib.pyplot as plt

# Título de la aplicación
st.title("Fragmento de Canción")

# Descargar la canción desde YouTube
st.write("Descargando la canción desde YouTube...")
link = "https://www.youtube.com/watch?v=544X2PIxCIc&ab_channel=vicentefernandezVEVO"

# Usar yt-dlp para obtener metadatos y descargar la canción
result = subprocess.run(['yt-dlp', '--extract-audio', '-o', 'audio', '--audio-format', 'mp3', '--print-json', link], capture_output=True, text=True)
video_info = json.loads(result.stdout)

# Extraer el nombre de la canción
song_title = video_info.get('title', 'Canción Desconocida')

# Mostrar el título de la canción
st.write(f"**Título de la canción descargada**: {song_title}")

# Convertir a formato WAV
st.write("Convirtiendo el archivo a formato WAV...")
subprocess.run(['ffmpeg', '-y', '-i', 'audio.mp3', 'output.wav'])

# Leer el archivo WAV
nombre_out = "output.wav"
x, fs = sf.read(nombre_out)

# Si el archivo tiene más de un canal, convertir a mono
if x.ndim > 1:
    x = np.mean(x, axis=1)

# Extraer los 5 segundos desde los 20 a los 25 segundos
start_sample = int(20 * fs)
end_sample = int(25 * fs)
x_fragment = x[start_sample:end_sample]

# Guardar el fragmento de audio como archivo WAV para que Streamlit pueda reproducirlo
fragment_filename = "fragment.wav"
sf.write(fragment_filename, x_fragment, fs)

# Mostrar información de frecuencia de muestreo y archivo
st.write(f"Frecuencia de muestreo: {fs} Hz")
st.write(f"Reproduciendo fragmento de 20 a 25 segundos de la canción:")

# Reproducir el fragmento en Streamlit
st.audio(fragment_filename)

# Parámetros de la portadora
Fc = 10000  # Frecuencia de la portadora
Im = st.slider("Seleccione el índice de modulación", 0.0, 1.0, 0.5)  # Índice de modulación

Fs = 5 * Fc  # Frecuencia de muestreo
t = np.arange(len(x_fragment)) / fs  # Vector de tiempo
m = x_fragment / np.max(np.abs(x_fragment))  # Normalizar el fragmento de audio -> Mensaje
Ac = np.max(np.abs(m)) / Im  # Amplitud de la portadora según índice de modulación
c = Ac * np.sin(2 * np.pi * Fc * t)  # Portadora

# Señal modulada en AM
y = (1 + m / Ac) * c

# Graficar la señal portadora y el mensaje
fig, ax = plt.subplots(figsize=(10, 8))
ax.plot(t, c, label='Portadora')
ax.plot(t, m, label='Mensaje')
ax.legend()
ax.set_xlabel('Tiempo [s]')
st.pyplot(fig)

# Graficar la señal modulada
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(t, y)
ax.set_xlabel('Tiempo [s]')
ax.set_ylabel('Señal Modulada AM')
ax.set_title('Modulación AM')
st.pyplot(fig)

# Espectro de la señal mensaje, portadora y modulada
Xfc = np.fft.fft(c)
Xfm = np.fft.fft(m)
Xfy = np.fft.fft(y)
vfre = np.fft.fftfreq(len(c), 1/Fs)

# Graficar los espectros con ajustes
fig, ax = plt.subplots(figsize=(6, 4))  # Ajustar el tamaño de la figura
ax.plot(vfre, np.abs(Xfy), label='Modulado')
ax.plot(vfre, np.abs(Xfm), label='Mensaje')
ax.plot(vfre, np.abs(Xfc), label='Portadora')
ax.legend()

# Ajustar los límites de los ejes
ax.set_xlim([-20000, 20000])  # Limitar el rango de frecuencias
ax.set_ylim([0, 7000])  # Limitar el rango de la magnitud

ax.set_xlabel('f [Hz]')
ax.set_ylabel('|X(f)|')
st.pyplot(fig)

# Guardar y reproducir los fragmentos de las señales moduladas
modulated_filename = "modulated.wav"
sf.write(modulated_filename, y, fs)

carrier_filename = "carrier.wav"
sf.write(carrier_filename, c, fs)

# Reproducir las señales
st.write("Señal del mensaje (fragmento original)")
st.audio(fragment_filename)

st.write("Señal portadora")
st.audio(carrier_filename)

st.write("Señal modulada AM")
st.audio(modulated_filename)


Overwriting pages/🎶_Pregunta_2.py


In [125]:
%%writefile pages/🎶_Pregunta_2_2.py
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, fftfreq
from scipy.signal import freqz
import streamlit as st

# Parámetros de la portadora
Fc = 10000  # Frecuencia de la portadora
Fs = 5 * Fc  # Frecuencia de muestreo
t = np.arange(len(x_fragment)) / fs  # vector de tiempo

# Señal modulada (DSB-SC)
y = (1 + m / Ac) * c  # La señal modulada que ya tienes

# Demodulación (mezcla con la portadora)
# Asumiendo θ = 0 como en la figura
y_demod = y * np.cos(2 * np.pi * Fc * t)  # Multiplicación con la portadora

# Diseño del filtro pasa bajas
cutoff = Fc / 2  # Frecuencia de corte del filtro (menor que la portadora)
order = 4  # Orden del filtro
b, a = signal.butter(order, cutoff / (Fs / 2), btype='low', analog=False)

# Aplicar el filtro pasa bajas a la señal demodulada
y_filtered = signal.filtfilt(b, a, y_demod)

# Escalamiento según la figura para recuperar la señal original
A1 = np.max(np.abs(y_filtered))  # Estimación de la amplitud de la señal modulada
m_recovered = 2 * y_filtered / A1  # Recuperamos el mensaje escalando adecuadamente

# Gráfica de polos y ceros del filtro pasa bajas
z, p, k = signal.tf2zpk(b, a)
fig1, ax1 = plt.subplots(figsize=(6, 6))
ax1.scatter(np.real(z), np.imag(z), marker='o', label='Ceros')
ax1.scatter(np.real(p), np.imag(p), marker='x', label='Polos')
ax1.set_xlabel('Parte Real')
ax1.set_ylabel('Parte Imaginaria')
ax1.set_title('Diagrama de Polos y Ceros del Filtro')
ax1.legend()
ax1.grid()

# Mostrar la primera gráfica en Streamlit
st.pyplot(fig1)

# Diagrama de Bode del filtro pasa bajas
w, h = freqz(b, a, fs=Fs)
fig2, (ax2, ax3) = plt.subplots(2, 1, figsize=(10, 6))
ax2.plot(w, 20 * np.log10(np.abs(h)))
ax2.set_title('Respuesta en Frecuencia del Filtro Pasa Bajas')
ax2.set_xlabel('Frecuencia [Hz]')
ax2.set_ylabel('Magnitud [dB]')
ax3.plot(w, np.angle(h))
ax3.set_xlabel('Frecuencia [Hz]')
ax3.set_ylabel('Fase [radianes]')

# Mostrar la segunda gráfica en Streamlit
st.pyplot(fig2)

# Gráfico del mensaje recuperado después de la demodulación
t = np.arange(len(m_recovered)) / fs
fig3, ax4 = plt.subplots(figsize=(10, 6))
ax4.plot(t, m_recovered)
ax4.set_xlabel('Tiempo [s]')
ax4.set_ylabel('Señal Demodulada (Mensaje Recuperado)')
ax4.set_title('Mensaje Recuperado Después de la Demodulación')

# Mostrar la tercera gráfica en Streamlit
st.pyplot(fig3)


Overwriting pages/🎶_Pregunta_2_2.py


In [136]:
#Detector de genero
%%writefile pages/🎶_Detector_De_Genero.py
import os
import numpy as np
import librosa
import librosa.display
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import streamlit as st

# Función para cargar canciones desde una carpeta
def load_songs_from_folder(folder_path, genres):
    X_train = []
    y_train = []

    # Recorrer cada género musical
    for genre in genres:
        genre_folder = os.path.join(folder_path, genre)

        # Verificar si la carpeta del género existe
        if not os.path.exists(genre_folder):
            st.error(f"No se encontró la carpeta para el género: {genre_folder}")
            continue

        # Recorrer los archivos de audio en la carpeta del género
        for filename in os.listdir(genre_folder):
            if filename.endswith(".wav"):  # Asegúrate de que sean archivos WAV
                filepath = os.path.join(genre_folder, filename)
                signal, sr = librosa.load(filepath, sr=None)  # Cargar el archivo de audio
                features = extract_features(signal, sr)  # Extraer características MFCC
                X_train.append(features)
                y_train.append(genre)  # Etiqueta según el género

    return np.array(X_train), np.array(y_train)

# Extraer características del audio (MFCCs)
def extract_features(signal, sr):
    mfccs = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13)
    return np.mean(mfccs.T, axis=0)

# Función para entrenar el modelo de clasificación de género musical
def train_genre_classifier(X_train, y_train):
    # Escalar las características
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    # Entrenar el modelo SVM
    clf = SVC(kernel='linear')
    clf.fit(X_train_scaled, y_train)

    return clf, scaler

# Clasificación del género musical
def classify_genre(clf, scaler, signal, fs):
    features_test = extract_features(signal, fs)
    features_test_scaled = scaler.transform([features_test])
    predicted_genre = clf.predict(features_test_scaled)
    return predicted_genre[0]

# Configurar Streamlit
st.title("Detector de Género Musical")

# Definir los géneros que queremos clasificar
genres = ['Ranchera', 'Reggae', 'Vallenato']  # Asegúrate de que estos géneros coincidan con las carpetas

# Subida del archivo de audio
uploaded_file = st.file_uploader("Sube un archivo de audio (formato .wav)", type=["wav"])

# Cargar las canciones para entrenar el modelo
music_folder = '/content'  # Ruta donde se han descargado las canciones en Colab
X_train, y_train = load_songs_from_folder(music_folder, genres)

if len(X_train) == 0:
    st.error("No se encontraron archivos de audio en la carpeta de entrenamiento.")
else:
    # Entrenar el modelo
    clf, scaler = train_genre_classifier(X_train, y_train)

    if uploaded_file is not None:
        # Cargar el archivo de prueba
        signal, sr = librosa.load(uploaded_file, sr=None)

        # Clasificar el género de la nueva canción
        predicted_genre = classify_genre(clf, scaler, signal, sr)

        # Mostrar el género detectado
        st.write(f"Género Detectado: **{predicted_genre}**")

        # Reproducir la canción subida
        st.audio(uploaded_file)


Overwriting pages/🎶_Detector_De_Genero.py


In [127]:
!ngrok authtoken 2lLlaYppwjUXEQ4Pp2l4E9NoElt_4huLNwmWHqjHmC5mHnBtt

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [128]:
from pyngrok import ngrok
import subprocess

# Ejecutar Streamlit en segundo plano
streamlit_command = 'streamlit run 1_📚_Parcial_3.py'
process = subprocess.Popen(streamlit_command.split())

# Exponer el servidor de Streamlit local a través de Ngrok
public_url = ngrok.connect('8501')
print(f"Aplicación de Streamlit disponible en: {public_url}")


Aplicación de Streamlit disponible en: NgrokTunnel: "https://baa7-34-168-234-110.ngrok-free.app" -> "http://localhost:8501"
